In [1]:
import torch
from torch import nn as nn
from models.nac import NAC
from models.nalu import NALU
from utilities.train_utils import get_eval_loss
from utilities.train_utils import get_eval_preds
from utilities.train_utils import train
from torch.optim import Adam
import numpy as np
import pandas as pd

### Function to create toy dataset

In [244]:
def make_data(min_val, max_val, num_obs, op):
    '''
    Generates toy data and target by sampling values from a
    uniform distribution parameterized by min_val and max_val.
    '''
#     np.random.seed(7)
    data = np.random.uniform(min_val, max_val, size=(num_obs, 2))
    if op == '+':
        targets = data[:, 0] + data[:, 1]
    elif op == '-':
        targets = data[:, 0] - data[:, 1]
    elif op == '*':
        targets = data[:, 0] * data[:, 1]
    elif op == '/':
        targets = data[:, 0] / data[:, 1]
    elif op == '^2':
        data = np.random.randint(min_val, max_val, size=(num_obs, 1)).astype(np.float32)
        targets = data ** 2
    elif op == 'sqrt':
        data = np.random.randint(min_val, max_val, size=(num_obs, 1))
        targets = np.sqrt(data)
    return data, targets

### Function to compute accuracy of predictions

In [3]:
def accuracy_score(preds, targets, tol=1e-3):
    '''
    Computes prediction accuracy by checking if
    predictions are equal to the target upto `tol`
    places after decimal.
    '''
    preds = preds.cpu().numpy().flatten()
    targets = targets.cpu().numpy().flatten()
    accuracy = np.isclose(preds, targets, rtol=tol)
    accuracy = accuracy.astype(np.int32).mean()
    return accuracy * 100

### Running experiments

In [5]:
eps = 1e-12
ops = ['+', '-', '*', '/']
test_scores = {}
models=[]
for op in ops:
        
    X_train, Y_train = make_data(10, 20, 10000, op)
    X_valid, Y_valid = make_data(10, 20, 200, op)
    X_test, Y_test = make_data(0, 30, 10000, op) # Test set contains both interpolated
                                                 # and extrapolated data

    model = train(NALU(X_train.shape[1], 1), nn.SmoothL1Loss(), Adam, X_train, Y_train,
                  X_valid, Y_valid, patience=15, batch_size=32,num_epochs = 1000)
    test_preds, test_targets = get_eval_preds(model, X_test,
                                              Y_test, 32, False)
    
    test_acc = accuracy_score(test_preds, test_targets)
    
    print('Test accuracy for [{}]: {:.2f}%'.format(op, test_acc))
    models.append(model)
    
    if op == 'sqrt':
        test_scores['sqrt(a)'] = test_acc
    else:
        test_scores['a '+op+' b'] = test_acc

Test accuracy for [+]: 100.00%


Test accuracy for [-]: 100.00%


Test accuracy for [*]: 100.00%


Test accuracy for [/]: 94.83%


### Saving the results

In [63]:
X_test, Y_test = make_data(10, 20, 1000, '+')
X_test = np.round(X_test)
Y_test = X_test[:,0] + X_test[:,1]

# test_preds, test_targets = get_eval_preds(models[0], X_test,
#                                               Y_test, 32, False)

X_test = torch.from_numpy(X_test).float()
preds = models[0].forward(X_test)

correct = 0
preds = preds.detach().numpy()
for i in range(X_test.shape[0]):
    if np.isclose(preds[i],Y_test[i]):
        correct+=1

test_acc = correct * 100.0 / X_test.shape[0]
print(test_acc)

100.0


In [293]:
X_test, Y_test = make_data(10, 20, 1000, '-')
X_test = np.round(X_test)
Y_test = X_test[:,0] - X_test[:,1]

# test_preds, test_targets = get_eval_preds(models[0], X_test,
#                                               Y_test, 32, False)

X_test = torch.from_numpy(X_test).float()
preds = models[1].forward(X_test)

correct = 0
preds = preds.detach().numpy()
for i in range(X_test.shape[0]):
    if np.isclose(preds[i],Y_test[i]):
        correct+=1

test_acc = correct * 100.0 / X_test.shape[0]
print(test_acc)

90.5


In [295]:
X_test, Y_test = make_data(0, 30, 1000, '*')
X_test = np.round(X_test)
Y_test = X_test[:,0] * X_test[:,1]

# test_preds, test_targets = get_eval_preds(models[0], X_test,
#                                               Y_test, 32, False)

X_test = torch.from_numpy(X_test).float()
preds = models[2].forward(X_test)

correct = 0
preds = preds.detach().numpy()
for i in range(X_test.shape[0]):
    if np.isclose(preds[i],Y_test[i]):
        correct+=1

test_acc = correct * 100.0 / X_test.shape[0]
print(test_acc)

97.2


In [317]:
X_test, Y_test = make_data(0, 30, 1000, '/')
X_test = np.round(X_test)
Y_test = X_test[:,0] / X_test[:,1]
Y_test = np.round(Y_test)
# test_preds, test_targets = get_eval_preds(models[0], X_test,
#                                               Y_test, 32, False)

X_test = torch.from_numpy(X_test).float()
preds = models[3].forward(X_test)

correct = 0
preds = preds.detach().numpy().round()
for i in range(X_test.shape[0]):
    if np.isclose(preds[i],Y_test[i]):
        correct+=1

test_acc = correct * 100.0 / X_test.shape[0]
print(test_acc)

C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


95.8


In [8]:
results = pd.DataFrame.from_dict(test_scores, orient='index', columns=['Accuracy'])
results.to_csv('results.csv', index=False)
results.head(4)

Accuracy
a + b    100.00
a - b     99.97
a * b    100.00
a / b     94.56